In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats as st
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
#import xesmf as xe
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#import cmocean.cm as cmo
import os
import glob
from sys import exit, argv
import netCDF4 as nc4
from distributed import Client
from h5py import File
from pyhdf.SD import SD, SDC
import re
from datetime import datetime

import matplotlib.pyplot as plt
import cartopy.io.shapereader as shpreader


In [2]:
#Path to data
data_path = '/Users/ashneelchandra/OneDrive\ -\ The\ University\ of\ Melbourne/Graduate\ Colloquium\ -\ Global\ Hackathon/hackathon-2025-australia-node/Fiji_Meteorological_Service/'


<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
/var/folders/59/qvr_15gx75gfflmn_7cl52f40000gn/T/ipykernel_84590/1962769547.py:2: SyntaxWarning: invalid escape sequence '\ '
  data_path = '/Users/ashneelchandra/OneDrive\ -\ The\ University\ of\ Melbourne/Graduate\ Colloquium\ -\ Global\ Hackathon/hackathon-2025-australia-node/Fiji_Meteorological_Service/'


In [ ]:
#Station locations (you can comment/uncomment the stations you do not need or need)

#Suva (Manual)
lat_station = -18.147571
lon_station = 178.453610

#Suva (AWS)
lat_station = -18.147549
lon_station = 178.453608

#Lautoka (Manual)
lat_station = -17.618600
lon_station = 177.438900

#Lautoka (TB3)
lat_station = -17.618897
lon_station = 177.438730

#Penang (TB3)
lat_station = -17.373833
lon_station = 178.171619

#Savusavu (Airport)
lat_station = -16.806141
lon_station = 179.342817

#Labasa (Airport)
lat_station = -16.468900
lon_station = 179.339700

In [ ]:
# Plot Fiji map and station locations

